In [2]:
%load_ext Cython

In [4]:
%%cython
import numpy as np
class Particula:#En esta clase definimos los métodos fundamentales que debe seguir cada partícula

    def __init__(self, (float,float) posicion , (float,float) velocidad , float masa ,float radio): 
       """comenzamos definiendo el método que debe 
        inicializar la clase, en este definimos los atributos básicos de la clase partícula
        las entradas masa y radio son de tipo float y las entradas de posición son tuplas conformadas por floats"""  
       self.radio=radio #Estas variables son de tipo entero  
       self.masa=masa 
       
       self.posicion=np.array(posicion)   #Estas variables son arrays de numpy, y las trataremos como vectores
       self.velocidad=np.array(velocidad)
       self.velocidad_mag=np.linalg.norm(self.velocidad)#Este atributo no es otra cosa que la magnitud del vector velocidad
     
       """Creamos dos listas con copias de los vectores posición y velocidad y velocidad_mag, posteriormente las usaremos
       para generar una lista con todos los valores que ha tenido la velocidad y posición"""
       
       self.velocidad_n=[self.velocidad]    ##Velocidad_n y posición_n son listas de vectores que contienen las listas correspondiente a
       self.posicion_n=[self.posicion]      ##todos los valores de estos vectores
       self.velocidad_mag_n=[self.velocidad_mag]
     
  ##########################################################################
    def paso_dt(self,float dt):   #Falta definir el valor que va a tomar dt  
       """Este método lo que hace es avanzar en el tiempo, cambia la posición de la partícula y agrega a las listas
       una actualización tanto de la posición de la partícula como la velocidad de la misma"""
      
       self.posicion=self.posicion + self.velocidad*dt  
       self.posicion_n.append(self.posicion) #Se añaden 
       self.velocidad_n.append(self.velocidad)
       self.velocidad_mag_n.append(self.velocidad_mag)
 ###############################################################################################
    def ver_colision_pp(self,otra_p):
       """Este método verifica si se dio luegar a una colisión entre dos partículas, las entradas
      son dos partículas, se definen sus radios y posiciones y se plantea una condición que indica 
      si las partículas chocaron"""
       cdef float r1=self.radio
       cdef float r2=otra_p.radio
       p1=self.posicion
       p2=otra_p.posicion
       cdef float sep=np.linalg.norm(p1-p2)#Norma del vector separación de ambas partículas.
       if sep-(r1+r2)<=0:#Si la separación es menor o igual a la suma de sus radios, entonces las partículas están en contacto y por lo tanto chocaron.
         return True
       else:
         return False
######################################################################################################
    def ver_colision_esquina(self, float Lx, float Ly):
       "Revisa si hay colisión con una esquina, Lx y Ly son las dimensiones horizontal y vertical de la caja respectivamente"
       if self.posicion[0] - self.radio >0 and self.posicion[1] - self.radio > 0:  #Esto corresponde a no chocar con la esquina (0,0)
         return False
       elif self.posicion[0] + self.radio <Lx and self.posicion[1] - self.radio > 0: #Esto corresponde a no chocar con la esquina (Lx,0)
         return False
       elif self.posicion[0] + self.radio <Lx and self.posicion[1] + self.radio <Ly:  #Esto corresponde a no chocar con la esquina (Lx,Ly)
         return False
       elif self.posicion[0] - self.radi0 > 0 and self.posicion[1] + self.radio <Ly:  #Esto corresponde a chocar con la esquina (0,Ly)
         return False
       else:
         return True
        
#####################################################################################################
    def ver_colision_muro(self, float Lx, float Ly):
        """Este método revisa si la partícula ha chocado contra un muro."""
        """La condición ve que si la posición en "x" es diferente 0 o la longitud horizontal (Lx) de la caja y adicionalmente
         #  si la posición en "y" es diferente de 0 o la longitud vertical (Ly) entonces la partícula no está chocando contra un muro"""
        if self.posicion[0] + self.radio <Lx and self.posicion[0] - self.radio!=0:
          return False
        elif self.posicion[1] - self.radio <Ly and self.posicion[1] - self.radio !=0:
          return False
        else:
          return True 
        
######################################################################################################
    #def resolver_colision_pared(self,otra_p):
     #   dt = 1 #Hay que definir el tamaño de dt
      #  self.paso_dt(dt)
        


In [5]:
p1 = Particula((0,0),(3,4),1,0.5)
for j in range(10):
  p1.paso_dt(1)
print(p1.posicion_n)

[array([0., 0.]), array([3., 4.]), array([6., 8.]), array([ 9., 12.]), array([12., 16.]), array([15., 20.]), array([18., 24.]), array([21., 28.]), array([24., 32.]), array([27., 36.]), array([30., 40.])]


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
fig, axes=plt.subplots()
x_datos=[]
y_datos=[]
graficar,=plt.plot([],[])

def lim():
  axes.set_xlim(0,2*np.pi)
  axes.set_ylim(-1,1)
  return graficar,

def funcion(x):
  x_datos.append(x)
  y_datos.append(np.cos(3*x))
  graficar.set_data(x_datos,y_datos)
  return graficar,

ani=FuncAnimation(fig,funcion,frames=np.linspace(0,4*np.pi,500),init_func=lim,blit=True)
from IPython.display import HTML
HTML(ani.to_jshtml())